# 线性回归Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 回归分析

回归分析是一种用于预测和推理的统计方法，通过样本数据学习**目标变量**与**自变量**之间的因果关系，建立数学模型。

例如，房屋售价$ y $会由一系列因素决定：面积$ x_1 $、地段$ x_2 $、房型$ x_3 $、是否为学区房$ x_4 $等。

回归分析的目标就是利用历史数据找出它们之间的映射关系，然后预测未来的值。

常用的回归方法包括：

- 线性回归Linear Regression
- 逻辑回归Logistic Regression
- 多项式回归Polynomial Regression

假设已知$ x $和$ f $，计算$ y = f(x) $是很容易的。

$ x = 2,\ f(x) = 3x + 5,\ y = 11 $

$ x = 2,\ f(x) = e^{sin(x)},\ y = 2.48 $

$ x = 2,\ f(x) = x^2 + 0.2x,\ y = 4.4 $

但是，如果知道$ x $和$ y $，算出$ f $会很困难。

![](./img/datapoint.png)

![](./img/linear.png)

![](./img/quadratic.png)

![](./img/exponential.png)

机器学习的目的就是从一系列可能的函数中找到最能匹配数据模式的函数。

## 线性回归

`salary.csv`中保存了一些人工作经验和薪资的数据。

In [ ]:
df = pd.read_csv('data/salary.csv')
df.head()

In [ ]:
plt.scatter(df['YearsExperience'], df['Salary'])
plt.title('工作经验与薪资的关系')
plt.xlabel('工作经验')
plt.ylabel('薪资')
plt.show()

通过绘制散点图可以发现，数据总体成线性关系。

`sklearn`提供了`LinearRegression`类用于创建线性回归模型。

In [ ]:
from sklearn.linear_model import LinearRegression

`fit()`用于将因变量和目标变量传入，该函数建立一个能够最佳满足数据趋势的线性函数模型。

In [ ]:
model = LinearRegression()
model.fit(df[['YearsExperience']], df['Salary'])

通过`intercept_`和`coef_`可以查看函数的**截距**和**系数**。

In [ ]:
model.intercept_, model.coef_

$ Salary = 9449.96 \times YearsExperience + 24848.20 $

通过这个模型，可以根据工作经验去预测薪资。

In [ ]:
df_test = pd.read_csv('data/salary_to_predict.csv')
df_test.head()

In [ ]:
model.predict(df_test[['YearsExperience']])

可以把预测的薪资添加到表中。

In [ ]:
df_test['PredictedSalary'] = model.predict(df_test[['YearsExperience']])
df_test.head()

In [ ]:
df_test.to_csv('data/salary_predicted.csv', index=False)

## 回归分析

误差是用来判断模型准确性的标准之一。

In [ ]:
line_x = np.linspace(df['YearsExperience'].min(), df['YearsExperience'].max(), 100)
line_x

In [ ]:
line_y = model.coef_ * line_x + model.intercept_
line_y

In [ ]:
plt.scatter(df['YearsExperience'], df['Salary'])
plt.plot(line_x, line_y, c='r', label='model')

plt.title('工作经验与薪资的关系')
plt.xlabel('工作经验')
plt.ylabel('薪资')
plt.legend()
plt.show()

### 平均绝对误差（MAE, MeanAbsoluteError）

MAE为预测值和实际值之间的绝对差异的平均值，它可以让我们了解预测的平均错误程度。

$$
MAE = {1 \over n} {\sum_{i=1}^{n} \left| y_i - \hat y_i\right|}
$$

$ n $为样本数量，$ y_i $为实际值，$ \hat y_i $为预测值。

In [ ]:
predicted_salary = model.predict(df[['YearsExperience']])
predicted_salary

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
MAE = mean_absolute_error(df['Salary'], predicted_salary)
MAE

### 均方误差（MSE, Mean Squared Error）

MSE为预测值和实际值之间差异的平方的平均值，MSE对较大误差的惩罚比MAW更大，因为误差被平方了。

$$
MSE = {1 \over n} {\sum_{i=1}^{n} \left( y_i - \hat y_i\right) ^ 2}
$$

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
MSE = mean_squared_error(df['Salary'], predicted_salary)
MSE

## 均方根误差（RMSE, Root Mean Squared Error）

RMSE是MSE的平方根，用于将误差指标转换回与原始数据相同的尺度。

$$
RMSE = \sqrt{{1 \over n} {\sum_{i=1}^{n} \left( y_i - \hat y_i\right) ^ 2}}
$$

In [ ]:
RMSE = np.sqrt(MSE)
RMSE

### 决定系数$ R^2 $

$ R^2 $是实际值与预测值差异平方和与实际值与其平均值差异平方和之比的补数，用于衡量模型对数据拟合的好坏。

$ R^2 $的取值范围为$ 0 \sim 1 $，表示目标变量的预测值和真实值之间的相关程度。$ R^2 $值越大，表示预测效果越好。

$$
R^2 = 1 - {{\sum_{i=1}^{n} \left( y_i - \hat y_i\right) ^ 2} \over {\sum_{i=1}^{n} \left( y_i - \bar y\right) ^ 2}}
$$

$ \bar y $为实际值的平均值。

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
R2 = r2_score(df['Salary'], predicted_salary)
R2

## 训练集/测试集

使用训练集和测试集是为了评估模型的**泛化**能力，如果只用训练集，我们无法准确地评估模型在未知数据上的表现。

训练集用来训练模型，让模型学习数据的特征和模式。测试集是模型之前未见过的数据，用来评估模型在新数据上的表现。

In [ ]:
df = pd.read_csv('data/salary.csv')
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['YearsExperience']], df['Salary'], test_size=0.2)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

![](./img/训练集测试集.png)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
r2_score(y_test, y_pred)

## 练习

### 广告收益预测

某公司为了推销产品，在电视、微博、微信等多种渠道投放广告。

目前，企业搜集了200条历史数据构成数据集，每条数据给出每个月3种渠道的广告投入（单位：万元），以及销量（单位：万个）。

#### 任务

公司预期下个月广告投放量为：
- 电视：130.1
- 微博：87.8
- 微信：69.2

建立一个能够很好用于预测未来销量的模型。

#### 读取数据

In [ ]:
df = pd.read_csv('data/advertising.csv', index_col=0)
df.head()

#### 数据分析

绘制**电视广告投入与销量的关系**图像：

In [ ]:
plt.scatter(df['TV'], df['Sales'])
plt.title('电视广告投入与销量的关系')
plt.xlabel('电视广告投入')
plt.ylabel('销量')
plt.show()

绘制**微博广告投入与销量的关系**的关系图像：

In [ ]:
plt.scatter(df['Weibo'], df['Sales'])
plt.title('微博广告投入与销量的关系')
plt.xlabel('微博广告投入')
plt.ylabel('销量')
plt.show()

绘制**微信广告投入与销量的关系**的关系图像：

In [ ]:
plt.scatter(df['WeChat'], df['Sales'])
plt.title('微信广告投入与销量的关系')
plt.xlabel('微信广告投入')
plt.ylabel('销量')
plt.show()

#### 建立模型

各平台的广告投入与销量大致呈线性关系，我们希望通过线性回归建立一个能够预测未来销量的模型。

![](./img/3D.jpg)

拆分训练集和测试集：

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['TV', 'Weibo', 'WeChat']], df['Sales'], test_size=0.2)

创建线性回归模型：

In [ ]:
model = LinearRegression()

训练模型：

In [ ]:
model.fit(X_train, y_train)

查看截距和系数：

In [ ]:
model.intercept_, model.coef_

$$
销量 = x_1电视 + x_2微博 + x_3微信 + b
$$

使用测试集进行预测：

In [ ]:
y_pred = model.predict(X_test)
y_pred

#### 评估模型

查看$ R^2 $评估模型的准确性：

In [ ]:
r2_score(y_test, y_pred)

#### 预测

预测下个月的销量：

In [ ]:
predicted_sales = model.predict(np.array([[131.8, 87.8, 69.2]]))
predicted_sales